In [ ]:
from sklearn.metrics import mean_absolute_error as MSE
from cmcrameri import cm
from tqdm import tqdm
import seaborn as sns
from common import *
from models import *

In [ ]:
df = pd.read_csv("../data/input_data/MAIN_DATASET.csv")

price = df['NO2_price'].values.reshape(-1,1)
fload = df['NO2_load_forecasted'].values.reshape(-1,1)
aload = df['NO2_load_actual'].values.reshape(-1,1)
fgen = df['NO2_generation_forecast'].values.reshape(-1,1)
agen = df['NO2_generation_actual'].values.reshape(-1,1)


price_days = seperate_column_to_days(price)
aload_days = seperate_column_to_days(aload)
fload_days = seperate_column_to_days(fload)
fgen_days = seperate_column_to_days(fgen)
agen_days = seperate_column_to_days(agen)

In [ ]:
lookbehind = 7
input_width = lookbehind*24
horizon = 24
no_hours = input_width + horizon
hour_in_days = int(no_hours / 24)

price_dataset = []
aload_dataset = []
fload_dataset = []
fgen_dataset = []
agen_dataset = []

for i in range(len(price_days) - hour_in_days+1):
    price_dataset.append(np.concatenate((price_days[i:i+hour_in_days])))
    aload_dataset.append(np.concatenate((aload_days[i:i+hour_in_days])))
    fload_dataset.append(np.concatenate((fload_days[i:i+hour_in_days])))
    fgen_dataset.append(np.concatenate((fgen_days[i:i+hour_in_days])))
    agen_dataset.append(np.concatenate((agen_days[i:i+hour_in_days])))

price_dataset = np.array(price_dataset)
aload_dataset = np.array(aload_dataset)
fload_dataset = np.array(fload_dataset)
fgen_dataset = np.array(fgen_dataset)
agen_dataset = np.array(agen_dataset)

scaler = MinMaxScaler()
price_dataset = scaler.fit_transform(price_dataset[:,:,0])
price_dataset = price_dataset[..., np.newaxis].astype(np.float32)

aload_dataset = scaler.fit_transform(aload_dataset[:,:,0])
aload_dataset = aload_dataset[..., np.newaxis].astype(np.float32)

fload_dataset = scaler.fit_transform(fload_dataset[:,:,0])
fload_dataset = fload_dataset[..., np.newaxis].astype(np.float32)

fgen_dataset = scaler.fit_transform(fgen_dataset[:,:,0])
fgen_dataset = fgen_dataset[..., np.newaxis].astype(np.float32)

agen_dataset = scaler.fit_transform(agen_dataset[:,:,0])
agen_dataset = agen_dataset[..., np.newaxis].astype(np.float32)

dataset_actual = np.concatenate((price_dataset, aload_dataset, agen_dataset), axis=2)
dataset_forecast = np.concatenate((price_dataset, fload_dataset, fgen_dataset), axis=2)

n,m,k = dataset_actual.shape

# Splitting the data, actual case

In [ ]:
train = int(0.7*n)
valid = int(0.9*n)
X_train = dataset_actual[:train, :input_width]
X_valid = dataset_actual[train:valid, :input_width]
X_test = dataset_actual[valid:, :input_width]

Y = np.empty((n, input_width, horizon))
for step_ahead in range(1, horizon + 1):
    Y[:,:, step_ahead - 1] = dataset_actual[:,step_ahead:step_ahead + input_width, 0]

Y_train = Y[:train]
Y_valid = Y[train:valid]
Y_test = Y[valid:]

# Grid search, first small then large architecture

In [ ]:
epochs = 10
no_neurons = np.array([16,32,64,128])
mse_list_small_actual = np.zeros((3, len(no_neurons)))
best_mse_small_actual = np.inf*np.ones(3)

In [ ]:
for n, neurons in enumerate(no_neurons):
    print(f"Iteration {n}")
    rnn = keras.models.Sequential([
        keras.layers.SimpleRNN(neurons, return_sequences=True, input_shape=[None, k]),
        keras.layers.Dense(horizon)
    ])

    lstm = keras.models.Sequential([
        keras.layers.LSTM(neurons, return_sequences=True, input_shape=[None, k]),
        keras.layers.Dense(horizon)
    ])

    gru = keras.models.Sequential([
        keras.layers.GRU(neurons, return_sequences=True, input_shape=[None, k]),
        keras.layers.Dense(horizon)
    ])

    rnn.compile(loss="mse", optimizer="adam")
    lstm.compile(loss="mse", optimizer="adam")
    gru.compile(loss="mse", optimizer="adam")

    rnn.fit(X_train, Y_train, epochs=epochs, validation_data=(X_valid, Y_valid), verbose=False)
    lstm.fit(X_train, Y_train, epochs=epochs, validation_data=(X_valid, Y_valid), verbose=False)
    gru.fit(X_train, Y_train, epochs=epochs, validation_data=(X_valid, Y_valid), verbose=False)

    Y_pred_rnn = rnn.predict(X_test)
    Y_pred_lstm = lstm.predict(X_test)
    Y_pred_gru = gru.predict(X_test)

    mse_rnn = MSE(Y_test[:,-1], Y_pred_rnn[:,-1])
    mse_lstm = MSE(Y_test[:,-1], Y_pred_lstm[:,-1])
    mse_gru = MSE(Y_test[:,-1], Y_pred_gru[:,-1])

    if mse_rnn < best_mse_small_actual[0]:
        print(f"Found new best SimpleRNN mse {mse_rnn} with {neurons} neurons")
        best_mse_small_actual[0] = mse_rnn
        fname = f"../data/models/multivariate_actual_small_rnn.h5"
        keras.models.save_model(rnn, fname)

    if mse_lstm < best_mse_small_actual[1]:
        print(f"Found new best LSTM mse {mse_lstm} with {neurons} neurons")
        best_mse_small_actual[1] = mse_lstm
        fname = f"../data/models/multivariate_actual_small_lstm.h5"
        keras.models.save_model(lstm, fname)
        

    if mse_gru < best_mse_small_actual[2]:
        print(f"Found new best GRU mse {mse_gru} with {neurons} neurons")
        best_mse_small_actual[2] = mse_gru
        fname = f"../data/models/multivariate_actual_small_gru.h5"
        keras.models.save_model(gru, fname)

    mse_list_small_actual[0,n] = mse_rnn
    mse_list_small_actual[1,n] = mse_lstm
    mse_list_small_actual[2,n] = mse_gru


In [ ]:
plt.figure(figsize=(15,10))
plt.plot(mse_list_small_actual[0], '.--', markersize=20, label="SimpleRNN")
plt.plot(mse_list_small_actual[1], '.--', markersize=20, label="LSTM")
plt.plot(mse_list_small_actual[2], '.--', markersize=20, label="GRU")
plt.xticks(ticks = np.arange(4), labels=no_neurons)
plt.xlabel(f"Number of neurons")
plt.ylabel(f"Mean Squared Error")
plt.grid()
plt.legend()
plt.savefig(f"../figures/multivariate_actual_small_architecture_grid_search.pdf")

In [ ]:
mse_list_actual = np.zeros((3, len(no_neurons), len(no_neurons)))
best_mse_actual = np.inf*np.ones(3)

In [ ]:
for n, neurons_first in enumerate(no_neurons):
    for m, neurons_second in enumerate(no_neurons):
        print(f"Iteration {n}{m}")
        rnn = keras.models.Sequential([
            keras.layers.SimpleRNN(neurons_first, return_sequences=True, input_shape=[None, k]),
            keras.layers.SimpleRNN(neurons_second, return_sequences=True),
            keras.layers.Dense(horizon)
        ])

        lstm = keras.models.Sequential([
            keras.layers.LSTM(neurons_first, return_sequences=True, input_shape=[None, k]),
            keras.layers.LSTM(neurons_second, return_sequences=True),
            keras.layers.Dense(horizon)
        ])

        gru = keras.models.Sequential([
            keras.layers.GRU(neurons_first, return_sequences=True, input_shape=[None, k]),
            keras.layers.GRU(neurons_second, return_sequences=True),
            keras.layers.Dense(horizon)
        ])

        rnn.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
        lstm.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
        gru.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])

        rnn.fit(X_train, Y_train, epochs=epochs, validation_data=(X_valid, Y_valid), verbose=False)
        lstm.fit(X_train, Y_train, epochs=epochs, validation_data=(X_valid, Y_valid), verbose=False)
        gru.fit(X_train, Y_train, epochs=epochs, validation_data=(X_valid, Y_valid), verbose=False)

        Y_pred_rnn = rnn.predict(X_test)
        Y_pred_lstm = lstm.predict(X_test)
        Y_pred_gru = gru.predict(X_test)

        mse_rnn = MSE(Y_test[:,-1], Y_pred_rnn[:,-1])
        mse_lstm = MSE(Y_test[:,-1], Y_pred_lstm[:,-1])
        mse_gru = MSE(Y_test[:,-1], Y_pred_gru[:,-1])

        if mse_rnn < best_mse_actual[0]:
            print(f"Found new best SimpleRNN mse {mse_rnn} with {neurons_first} and {neurons_second} neurons")
            best_mse_actual[0] = mse_rnn
            fname = f"../data/models/multivariate_actual_large_rnn.h5"
            keras.models.save_model(rnn, fname)

        if mse_lstm < best_mse_actual[1]:
            print(f"Found new best LSTM mse {mse_lstm} with {neurons_first} and {neurons_second} neurons")
            best_mse_actual[1] = mse_lstm
            fname = f"../data/models/multivariate_actual_large_lstm.h5"
            keras.models.save_model(lstm, fname)

        if mse_gru < best_mse_actual[2]:
            print(f"Found new best GRU mse {mse_gru} with {neurons_first} and {neurons_second} neurons")
            best_mse_actual[2] = mse_gru
            fname = f"../data/models/multivariate_actual_large_gru.h5"
            keras.models.save_model(gru, fname)

        mse_list_actual[0,n,m] = mse_rnn
        mse_list_actual[1,n,m] = mse_lstm
        mse_list_actual[2,n,m] = mse_gru

In [ ]:
print(mse_list_actual)
print(best_mse_actual)

In [ ]:
plt.figure(figsize=(12,10))
gridsearch = sns.heatmap(mse_list_actual[0],annot=True, annot_kws={'fontsize':'xx-large'}, fmt=".5f", xticklabels= no_neurons, yticklabels= no_neurons, cmap=cm.lajolla)
gridsearch.set_yticklabels(gridsearch.get_yticklabels(),rotation = 0)

plt.title("SimpleRNN architecture GridSearch")
plt.xlabel("Neurons Second Layer")
plt.ylabel("Neurons First Layer")
plt.savefig(f"../figures/Large_actual_gridsearch_rnn.pdf")

plt.figure(figsize=(12,10))
gridsearch = sns.heatmap(mse_list_actual[1],annot=True, annot_kws={'fontsize':'xx-large'}, fmt=".5f", xticklabels= no_neurons, yticklabels= no_neurons, cmap=cm.lajolla)
gridsearch.set_yticklabels(gridsearch.get_yticklabels(),rotation = 0)

plt.title("LSTM architecture GridSearch")
plt.xlabel("Neurons Second Layer")
plt.ylabel("Neurons First Layer")
plt.savefig(f"../figures/Large_actual_gridsearch_lstm.pdf")

plt.figure(figsize=(12,10))
gridsearch = sns.heatmap(mse_list_actual[2],annot=True, annot_kws={'fontsize':'xx-large'}, fmt=".5f", xticklabels= no_neurons, yticklabels= no_neurons, cmap=cm.lajolla)
gridsearch.set_yticklabels(gridsearch.get_yticklabels(),rotation = 0)

plt.title("GRU architecture GridSearch")
plt.xlabel("Neurons Second Layer")
plt.ylabel("Neurons First Layer")
plt.savefig(f"../figures/Large_actual_gridsearch_gru.pdf")

# Splitting the data, forecast case

In [ ]:
n,m,k = dataset_forecast.shape

train = int(0.7*n)
valid = int(0.9*n)
X_train = dataset_forecast[:train, :input_width]
X_valid = dataset_forecast[train:valid, :input_width]
X_test = dataset_forecast[valid:, :input_width]

Y = np.empty((n, input_width, horizon))
for step_ahead in range(1, horizon + 1):
    Y[:,:, step_ahead - 1] = dataset_forecast[:,step_ahead:step_ahead + input_width, 0]

Y_train = Y[:train]
Y_valid = Y[train:valid]
Y_test = Y[valid:]

In [ ]:
epochs = 10
no_neurons = np.array([16,32,64,128])
mse_list_small_forecast = np.zeros((3, len(no_neurons)))
best_mse_small_forecast = np.inf*np.ones(3)

In [ ]:
for n, neurons in enumerate(no_neurons):
    print(f"Iteration {n}")
    rnn = keras.models.Sequential([
        keras.layers.SimpleRNN(neurons, return_sequences=True, input_shape=[None, k]),
        keras.layers.Dense(horizon)
    ])

    lstm = keras.models.Sequential([
        keras.layers.LSTM(neurons, return_sequences=True, input_shape=[None, k]),
        keras.layers.Dense(horizon)
    ])

    gru = keras.models.Sequential([
        keras.layers.GRU(neurons, return_sequences=True, input_shape=[None, k]),
        keras.layers.Dense(horizon)
    ])

    rnn.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
    lstm.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
    gru.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])

    rnn.fit(X_train, Y_train, epochs=epochs, validation_data=(X_valid, Y_valid), verbose=False)
    lstm.fit(X_train, Y_train, epochs=epochs, validation_data=(X_valid, Y_valid), verbose=False)
    gru.fit(X_train, Y_train, epochs=epochs, validation_data=(X_valid, Y_valid), verbose=False)

    Y_pred_rnn = rnn.predict(X_test)
    Y_pred_lstm = lstm.predict(X_test)
    Y_pred_gru = gru.predict(X_test)

    mse_rnn = MSE(Y_test[:,-1], Y_pred_rnn[:,-1])
    mse_lstm = MSE(Y_test[:,-1], Y_pred_lstm[:,-1])
    mse_gru = MSE(Y_test[:,-1], Y_pred_gru[:,-1])

    if mse_rnn < best_mse_small_forecast[0]:
        print(f"Found new best SimpleRNN mse {mse_rnn} with {neurons} neurons")
        best_mse_small_forecast[0] = mse_rnn
        fname = f"../data/models/multivariate_forecast_small_rnn.h5"
        keras.models.save_model(rnn, fname)

    if mse_lstm < best_mse_small_forecast[1]:
        print(f"Found new best LSTM mse {mse_lstm} with {neurons} neurons")
        best_mse_small_forecast[1] = mse_lstm
        fname = f"../data/models/multivariate_forecast_small_lstm.h5"
        keras.models.save_model(lstm, fname)
        

    if mse_gru < best_mse_small_forecast[2]:
        print(f"Found new best GRU mse {mse_gru} with {neurons} neurons")
        best_mse_small_forecast[2] = mse_gru
        fname = f"../data/models/multivariate_forecast_small_gru.h5"
        keras.models.save_model(gru, fname)

    mse_list_small_forecast[0,n] = mse_rnn
    mse_list_small_forecast[1,n] = mse_lstm
    mse_list_small_forecast[2,n] = mse_gru


In [ ]:
plt.figure(figsize=(15,10))
plt.plot(mse_list_small_forecast[0], '.--', markersize=20, label="SimpleRNN")
plt.plot(mse_list_small_forecast[1], '.--', markersize=20, label="LSTM")
plt.plot(mse_list_small_forecast[2], '.--', markersize=20, label="GRU")
plt.xticks(ticks = np.arange(4), labels=no_neurons)
plt.xlabel(f"Number of neurons")
plt.ylabel(f"Mean Squared Error")
plt.grid()
plt.legend()
plt.savefig(f"../figures/multivariate_forecast_small_architecture_grid_search.pdf")

In [ ]:
mse_list_forecast = np.zeros((3, len(no_neurons), len(no_neurons)))
best_mse_forecast = np.inf*np.ones(3)

In [ ]:
for n, neurons_first in enumerate(no_neurons):
    for m, neurons_second in enumerate(no_neurons):
        print(f"Iteration {n}{m}")
        rnn = keras.models.Sequential([
            keras.layers.SimpleRNN(neurons_first, return_sequences=True, input_shape=[None, k]),
            keras.layers.SimpleRNN(neurons_second, return_sequences=True),
            keras.layers.Dense(horizon)
        ])

        lstm = keras.models.Sequential([
            keras.layers.LSTM(neurons_first, return_sequences=True, input_shape=[None, k]),
            keras.layers.LSTM(neurons_second, return_sequences=True),
            keras.layers.Dense(horizon)
        ])

        gru = keras.models.Sequential([
            keras.layers.GRU(neurons_first, return_sequences=True, input_shape=[None, k]),
            keras.layers.GRU(neurons_second, return_sequences=True),
            keras.layers.Dense(horizon)
        ])

        rnn.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
        lstm.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
        gru.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])

        rnn.fit(X_train, Y_train, epochs=epochs, validation_data=(X_valid, Y_valid), verbose=False)
        lstm.fit(X_train, Y_train, epochs=epochs, validation_data=(X_valid, Y_valid), verbose=False)
        gru.fit(X_train, Y_train, epochs=epochs, validation_data=(X_valid, Y_valid), verbose=False)

        Y_pred_rnn = rnn.predict(X_test)
        Y_pred_lstm = lstm.predict(X_test)
        Y_pred_gru = gru.predict(X_test)

        mse_rnn = MSE(Y_test[:,-1], Y_pred_rnn[:,-1])
        mse_lstm = MSE(Y_test[:,-1], Y_pred_lstm[:,-1])
        mse_gru = MSE(Y_test[:,-1], Y_pred_gru[:,-1])

        if mse_rnn < best_mse_forecast[0]:
            print(f"Found new best SimpleRNN mse {mse_rnn} with {neurons_first} and {neurons_second} neurons")
            best_mse_forecast[0] = mse_rnn
            fname = f"../data/models/multivariate_forecast_large_rnn.h5"
            keras.models.save_model(rnn, fname)

        if mse_lstm < best_mse_forecast[1]:
            print(f"Found new best LSTM mse {mse_lstm} with {neurons_first} and {neurons_second} neurons")
            best_mse_forecast[1] = mse_lstm
            fname = f"../data/models/multivariate_forecast_large_lstm.h5"
            keras.models.save_model(lstm, fname)

        if mse_gru < best_mse_forecast[2]:
            print(f"Found new best GRU mse {mse_gru} with {neurons_first} and {neurons_second} neurons")
            best_mse_forecast[2] = mse_gru
            fname = f"../data/models/multivariate_forecast_large_gru.h5"
            keras.models.save_model(gru, fname)

        mse_list_forecast[0,n,m] = mse_rnn
        mse_list_forecast[1,n,m] = mse_lstm
        mse_list_forecast[2,n,m] = mse_gru

In [ ]:
print(mse_list_forecast)
print(best_mse_forecast)

In [ ]:
fig = plt.figure(figsize=(12,10))
gridsearch = sns.heatmap(mse_list_forecast[0],annot=True, annot_kws={'fontsize':'xx-large'}, fmt=".5f", xticklabels= no_neurons, yticklabels= no_neurons, cmap=cm.lajolla)
gridsearch.set_yticklabels(gridsearch.get_yticklabels(),rotation = 0)

plt.title("SimpleRNN architecture GridSearch")
plt.xlabel("Neurons Second Layer")
plt.ylabel("Neurons First Layer")
plt.savefig(f"../figures/Large_forecast_gridsearch_rnn.pdf")

fig = plt.figure(figsize=(12,10))
gridsearch = sns.heatmap(mse_list_forecast[1],annot=True, annot_kws={'fontsize':'xx-large'}, fmt=".5f", xticklabels= no_neurons, yticklabels= no_neurons, cmap=cm.lajolla)
gridsearch.set_yticklabels(gridsearch.get_yticklabels(),rotation = 0)

plt.title("LSTM architecture GridSearch")
plt.xlabel("Neurons Second Layer")
plt.ylabel("Neurons First Layer")
plt.savefig(f"../figures/Large_forecast_gridsearch_lstm.pdf")

plt.figure(figsize=(12,10))
gridsearch = sns.heatmap(mse_list_forecast[2],annot=True, annot_kws={'fontsize':'xx-large'}, fmt=".5f", xticklabels= no_neurons, yticklabels= no_neurons, cmap=cm.lajolla)
gridsearch.set_yticklabels(gridsearch.get_yticklabels(),rotation = 0)

fig = plt.title("GRU architecture GridSearch")
plt.xlabel("Neurons Second Layer")
plt.ylabel("Neurons First Layer")
plt.savefig(f"../figures/Large_forecast_gridsearch_gru.pdf")